# Analysis and Visualization of E3SM Data using UXarray

E3SM Tutorial Workshop 2024

05/07/2024

Authors: [Tom Vo](https://github.com/tomvothecoder) and [Stephen Po-Chedley](https://github.com/pochedls)


## Overview

This exercise notebook will walk you through the core UXarray data models and a few
analysis and visualization features to help you gain practical hands-on experience. Please visit the [UXarray documentation](https://uxarray.readthedocs.io/en/stable/) for more information on all of the available features.

### Exercises

1. Open E3SM Data with Grid Files
2. View Grid Information
3. Visualize Grid Topology
4. Face Area Calculations

If you are interested, more exercises can be found in the [UXarray Usage Examples](https://uxarray.readthedocs.io/en/stable/examples.html)!

### Helpful Prequisite Knowledge

If you are unfamiliar and interested in the topics below, click the links for
more information.

- [Differences between structured and unstructured grids](https://github.com/ProjectPythia/unstructured-grid-viz-cookbook/blob/main/notebooks/01-intro/01-unstructured-grid-overview.ipynb)
- [Data Mapping](https://github.com/ProjectPythia/unstructured-grid-viz-cookbook/blob/main/notebooks/01-intro/03-data-mapping.ipynb)
- [Plotting Libraries](https://github.com/ProjectPythia/unstructured-grid-viz-cookbook/blob/main/notebooks/02-methods/01-plotting-libraries.ipynb)
- [Rendering Techniques](https://github.com/ProjectPythia/unstructured-grid-viz-cookbook/blob/main/notebooks/02-methods/02-rendering-techniques.ipynb)

### Resources

- [UXarray Documentation](https://uxarray.readthedocs.io/en/stable/)
- This notebook was adapted from the [UXarray Usage Examples](https://uxarray.readthedocs.io/en/stable/examples.html) and the [Project Pythia Notebooks](https://projectpythia.org/unstructured-grid-viz-cookbook/README.html).


## Prerequisite: Set up the E3SM Unified Environment v1.10.0 Python Kernel

1. Open a terminal in Jupyter Hub.
2. Run the commands below to add the kernel to NERSC Jupyter Hub.

   ```bash
   source /global/common/software/e3sm/anaconda_envs/load_latest_e3sm_unified_pm-cpu.sh

   python -m ipykernel install \
   --user --name e3sm_unified_1.10.0 --display-name e3sm_unified.1.10.0
   ```

   &mdash; <cite>https://docs.nersc.gov/services/jupyter/how-to-guides/#how-to-use-a-conda-environment-as-a-python-kernel</cite>

3. Select the kernel for this Notebook by clicking the current kernel in the top-right
   (where it says NERSC Python in the screenshot).

   <img src="kernel-instructions-1.png" width=500px/>

4. Select `e3sm_unified.1.10.0` from the list of environments.

   <img src="kernel-instructions-2.png" width=500px/>


## Setup Code


In [14]:
import glob
import os

import numpy as np
import xarray as xr
import uxarray as ux

# The data directory containing the NetCDF files.
data_dir = (
    "/global/cfs/cdirs/e3sm/www/Tutorials/2024/simulations"
    "/extendedOutput.v3.LR.historical_0101/archive/atm/hist"
)
# The absolute paths to each NetCDF file in the data directory.
glob_path = os.path.join(data_dir, "*.nc")
data_paths = glob.glob(glob_path)

# The path to the grid file.
# FIXME is this the right grid file?
grid_path = "/global/cfs/cdirs/e3sm/diagnostics/grids/ne120pg2.nc"

## What are `ux.Dataset`, `ux.DataArray`, and `ux.Grid` objects?

- A [ux.UxDataset](https://uxarray.readthedocs.io/en/stable/user_api/generated/uxarray.UxDataset.html#uxarray.UxDataset) object is an `xarray.Dataset-like`, multi-dimensional, in memory, array database. This object inherits from `xarray.Dataset` and has its own unstructured grid-aware dataset operators and attributes through the `uxgrid` accessor.
- A [ux.UxDataArray](https://uxarray.readthedocs.io/en/stable/user_api/generated/uxarray.UxDataArray.html) object is an N-dimensional `xarray.DataArray-like` array. It inherits from `xarray.DataArray` and has its own unstructured grid-aware array operators and attributes through the `uxgrid` accessor.
- A [ux.Grid](https://uxarray.readthedocs.io/en/stable/user_api/generated/uxarray.Grid.html#) object represents a two-dimensional unstructured grid encoded following the UGRID conventions and provides grid-specific functionality.
  - Can be used standalone to work with unstructured grids, or can be paired with either a `ux.UxDataArray` or `ux.UxDataset` and accessed through the `.uxgrid` attribute.
  - For constructing a grid from non-UGRID datasets or other types of supported data, see our `ux.open_grid` method or specific class methods (`Grid.from_dataset`, `Grid.from_face_verticies`, etc.)

&mdash; <cite>https://uxarray.readthedocs.io/en/latest/getting-started/overview.html</cite>


### Exercise 1. Open E3SM Dataset with Grid Files using UXarray

When working with Unstructured Grids, the grid definition and data variables are often stored as separate files. This means that there are multiple separate files that need to be read and linked together to represent the entire dataset.

&mdash; <cite>https://uxarray.readthedocs.io/en/latest/examples/001-working-with-unstructured-grids.html#</cite>


#### 💻 Your turn:

Use `ux.open_mfdataset()` to open the grid file and the NetCDF files as a `ux.Dataset` object.

Hint: Use `grid_path` and `data_paths` as function arguments.


In [11]:
# Your code here. When ready, click on the three dots below for the solution.

In [16]:
uxds = ux.open_mfdataset(grid_path, data_paths[0:1])

Access the `TS` variable by indexing the `UxDataset` object to obtain a `UxDataArray` object.


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
uxds["TS"]

Extract the `ux.Grid` object from `uxds` and view the output.


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
grid = uxds.uxgrid
grid

### Exercise 2: Viewing Grid Information

For this exercise, just execute the cells below to view the outputs.

&mdash; <cite>https://uxarray.readthedocs.io/en/stable/examples/002-grid-topology.html</cite>


#### Grid Attributes

If our input grid contained additional attributes that were not representable by the UGRID conventions, they would be stored here


In [ ]:
grid.parsed_attrs

#### Grid Coordinates

The coordinates by default are represented in terms of longitude and latitude.


In [ ]:
grid.node_lon

In [ ]:
grid.node_lat

If you wish to use the Cartesian coordinate system, you can access the following attributes, which will internally construct a set of Cartesian coordinates derived from the previous set.


In [ ]:
grid.node_x

In [ ]:
grid.node_y

In [ ]:
grid.node_z

#### Grid Connectivity

Connectivity variables are used to describe how various geometric elements (nodes, faces, edges) can be manipulated and interconnected to represent the topology of the unstructured grid.

As described in the UGRID conventions, these connectivity variables are stored as integer arrays and may contain a Fill Value. UXarray standardizes both of these at the data loading step, meaning that the data type and fill value can always be guaranteed to be the following:


In [ ]:
ux.INT_DTYPE

In [ ]:
ux.INT_FILL_VALUE

Below we can see how to access these connectivity variables.


In [ ]:
grid.face_node_connectivity

In [ ]:
grid.n_nodes_per_face

### Exercise 3: Visualize the Grid Topology

In this exercise, we will visualize the topology of an unstructured grid (i.e., the
elements that make up a grid).

&mdash; <cite>https://uxarray.readthedocs.io/en/latest/examples/006-plot-api-topology.html.</cite>


#### Using the `Grid.plot()` Accessor

Each Grid object is initialized with a plotting accessor, which enables plotting routines to be called directly on the object. By default, calling `.plot()` on a `Grid` instance plots all the edges of a grid.

All of the plotting methods are built around the Holoviews package, so you can select between Matplotlib and Bokeh backends if desired (Bokeh is the default and is suggested).


#### 💻 Your turn:

Extract the grid topology from the `uxds` and plot it.

Hint: Use the `.uxgrid` attribute and call `.plot()`


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [4]:
grid.plot(title="Default Grid Plot Method", height=350, width=700)

NameError: name 'grid' is not defined

### Exercise 4: Face Area Calculations

This section covers the different area calculation options provided by `uxarray`.
Note, this is a only subset of the available options.

&mdash; <cite>https://uxarray.readthedocs.io/en/latest/examples/003-area-calc.html</cite>


#### 💻 Your turn:

Calculate the total face area for the grid.

Hint: Use `.calculate_total_face_area()`


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
t4_area = grid.calculate_total_face_area()
t4_area

Calculate the total face area using the Quadratre Rule and Order of 4.

Order:

```
   1 to 10              for gaussian
   1, 4, 8, 10 and 12   for triangular
```


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
t1_area = grid.calculate_total_face_area(quadrature_rule="triangular", order=1)

View the individual face areas using `Grid.face_areas`.


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
grid.face_areas

Calculate the area using `Grid.compute_face_areas`.


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
all_face_areas, all_face_jacobians = grid.compute_face_areas(
    quadrature_rule="gaussian", order=4
)
g4_area = all_face_areas.sum()
g4_area

Now we compare the values with actual know value and report error for each of the three cases above.

Just execute the cell below to view the outputs.


In [ ]:
actual_area = 4 * np.pi
diff_t4_area = np.abs(t4_area - actual_area)
diff_t1_area = np.abs(t1_area - actual_area)
diff_g4_area = np.abs(g4_area - actual_area)

diff_t1_area, diff_t4_area, diff_g4_area

## Visualizing E3SM Data as Polygons

Polygon plotting is the primary meethod for visualizing face-centered data variables in UXarray.

 <div class="alert alert-block alert-info">
<b>Info:</b> UXarray’s Plotting API is built around the <a href="https://holoviews.org/">Holoviews</a> package. For details about customization and accepted parameters, pleases refer to their documentation.
</div>


In [17]:
uxds["TREFHT"].isel(time=0).plot.polygons(line_width=0.1, title="Vector Polygon Plot")

/global/u2/v/vo13/mambaforge/envs/xcdat_test_uxarray/lib/python3.12/site-packages/uxarray/plot/dataarray_plot.py:392: UserWarning: Including Antimeridian Polygons may lead to visual artifacts. It is suggested to keep 'exclude_antimeridian' set to True.
  warnings.warn(


ValueError: Data Variable with size 21600 does not match the number of faces (345600.

## Interoperability with xCDAT

Since `ux.UxDataset` and `ux.UxDataArray` extend the `xr.Dataset` and `xr.DataArray` classes,
_most_ xCDAT APIs are interoperable with UXarray objects.

- The exception is xCDAT's [spatial averager](https://xcdat.readthedocs.io/en/latest/generated/xarray.Dataset.spatial.average.html), which requires data on rectilinear grids. The data must first be remapped from unstructured to rectilinear grid using another tool like `nco`.
- There are plans to support unstructured to structured regridding in UXarray in the future.

Resources:

- [xCDAT Documentation Homepage](https://xcdat.readthedocs.io/en/stable/)
- [xCDAT API Reference Guide](https://xcdat.readthedocs.io/en/stable/api.html)

## Next Steps

Feel free to jump over to the `xcdat_practicum_notebook.ipynb` to work with `nco` and `xcdat`.
